In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astroquery.mast import Observations
import lightkurve as lk
import os

In [2]:
#Url Pater
url = "/home/pater/Desktop/CosasPater/UNAL/2023-02NovenoSemestre/Exoplanet_IA/DataProyect/"

In [3]:
#Url Ari
url = "C:/Users/ASUS/OneDrive/Documentos/2023-2/Inteligencia/Proyecto/Curvas_de_luz/"

In [4]:
def descargar_curvas_de_luz(kepler_id,url):
    # Buscar las curvas de luz de un objetivo específico
    obs = lk.search_lightcurve(kepler_id, author='Kepler')
    lc = obs[-1].download()
    df = pd.DataFrame({'time': lc.time.value,  # Tiempo
                       'flux': lc.flux,  # Flujo
                       'flux_err': lc.flux_err  # Error en el flujo
                      })
    df.to_csv(f'{url}LC_{kepler_id}.csv', index=False)

In [3]:
Star_info = pd.read_csv('cumulative.csv')
Star_info

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,9560,10031643,K07984.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,0,0,...,-152.0,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
9560,9561,10090151,K07985.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,1,...,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
9561,9562,10128825,K07986.01,NaN,CANDIDATE,CANDIDATE,0.497,0,0,0,...,-220.0,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
9562,9563,10147276,K07987.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.021,0,0,1,...,-236.0,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385


In [4]:
contenido = os.listdir(url)
L_Str = []
for Str in contenido:
    Str = int(Str[3:-4])
    L_Str.append(Str)

In [10]:
KEPID = Star_info[:]['kepid']
i = 0
for name in KEPID:
    if name in L_Str:
        i += 1
        continue
    else:
        Id = str(name)
        descargar_curvas_de_luz(Id,url)
        continue

In [11]:
i

9564

In [5]:
Id = Star_info['kepid']
Id.to_csv(f'/home/pater/Desktop/CosasPater/UNAL/2023-02NovenoSemestre/Exoplanet_IA/Data_Id.csv', index=False)

In [10]:
import requests
from bs4 import BeautifulSoup

url = "https://archive.stsci.edu/kepler/kic10/search.php"

# Define los parámetros de búsqueda
payload = {
    'kic_kepler_id': '10797460',  # Reemplaza con el Kepler ID que estás buscando
    'action': 'Search'
}

response = requests.post(url, data=payload)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    # Aquí puedes procesar el contenido HTML para extraer los datos que necesitas
    tables = soup.find_all('table')
    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            for column in columns:
                print(column.get_text())
else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")







Mission Search / 
     	Missions / 
     	Contacts / 
     	STScI / 
     	MAST



Columns Help / 
		Archive Status



KIC10 Search Results










for the MAST table display.

number of rows returned = 1



Kepler ID
RA (J2000)
Dec (J2000)
r Mag
J Mag
Kepler Mag
2MASS ID
Star/Gal ID
Teff (deg K)
Log G (cm/s/s)
Metallicity (solar=0.0)
E(B-V)
Radius (solar=1.0)
Parallax (arcsec)
Total PM (arcsec/yr)
g-r color

 10797460
19 27 44.215
+48 08 29.94
15.270
14.082
15.347
19274421+4808299
0
5584
4.406
0.136
0.142
1.067
 
0.000
0.620



Kepler ID
RA (J2000)
Dec (J2000)
r Mag
J Mag
Kepler Mag
2MASS ID
Star/Gal ID
Teff (deg K)
Log G (cm/s/s)
Metallicity (solar=0.0)
E(B-V)
Radius (solar=1.0)
Parallax (arcsec)
Total PM (arcsec/yr)
g-r color




10797460
19 27 44.215
+48 08 29.94
15.270
14.082
15.347
19274421+4808299
0
5584
4.406
0.136
0.142
1.067
 
0.000
0.620
10797460
19 27 44.215
+48 08 29.94
15.270
14.082
15.347
19274421+4808299
0
5584
4.406
0.136
0.142
1.067
 
0.000
0.620
10797460
19 2